In [ ]:
# run through all files, open the file, and check if there are *any* data in our lat/lon rannge
L3files = glob.glob('downloads/RSS_smap_SSS_L3*') # use L3 SMAP - requires having one file stored locally


# loads a single data file ...
ds = xr.open_mfdataset(outfile, engine='h5netcdf', phony_dims='sort')
# phony_dim_0 is swath x, phony_dim_1 is swath y, phony_dim_2 is related to SMAP ambiguity, we can ignore this here
ds = ds.rename_dims({'phony_dim_0':'swath_x', 'phony_dim_1':'swath_y'})

In [ ]:
# check if there are lon/lat data in our domain:
lons = np.reshape(ds.lon.values,[1,-1])
lats = np.reshape(ds.lat.values,[1,-1])
sss = np.reshape(ds.smap_sss.values,[1,-1])

i = ((lons > lonrange[0]) & (lons < lonrange[1]) & (lats > latrange[0]) & (lats < latrange[1])) 
sss = sss[i]


In [ ]:
# follow severine's example to regrid & subsample
L3files = glob.glob('downloads/RSS_smap_SSS_L3*') # use L3 SMAP - requires having one file stored locally
ds_grid = xr.open_dataset(L3files[0])

# define grid lat/lon mesh
longrid = ds_grid.lon.values
latgrid = ds_grid.lat.values
LOgrid, LAgrid = np.meshgrid(longrid, latgrid)
print(LOgrid.shape,LAgrid.shape)

# use pyresample to define a target grid
target = pr.SwathDefinition(LOgrid, LAgrid)

# and the source data
lons = ds.lon.values
lats = ds.lat.values 
source = pr.SwathDefinition(lons, lats)

In [ ]:
# resample:
sss = ds.smap_sss.values
result, stddev, counts = resample_gauss(
    source,
    sss,
    target,
    radius_of_influence=175000,  
    sigmas=25000,
    neighbours=100,
    fill_value=np.nan,
    with_uncert=True,
)
result.shape

In [ ]:
# find non-nan data:
# sss = ds.smap_sss

# Severine's code identifies fill value, but the L2 SMAP data doesn't have this field
# - instead, should prob use quality code :/
# attrs = {}
# for k, v in ds.smap_sss.attrs.items():
#         if isinstance(v, bytes):
#             attrs[k] = v.decode('utf-8')
#         else:
#             attrs[k] = v
#         print(k, ' ', attrs[k])
# # attrs['_FillValue']
# sss = numpy.where(ds.smap_sss < attrs['_FillValue'], ds.smap_sss, numpy.nan)
sss = np.where(ds.smap_sss > 0, ds.smap_sss, np.nan)
nnans = ~np.isnan(sss)
sss = sss[nnans]

lons = ds.lon.values[nnans]
lats = ds.lat.values[nnans]  
lons = (lons + 180) % 360 - 180
print(sss.shape, lons.shape, lats.shape)

In [ ]:
dumx = np.where((ds.lon<lonrange[0], ds.lon>lonrange[1]), ds.lon, np.nan)
dumy = np.where((ds.lat<latrange[0], ds.lat>latrange[1]), ds.lat, np.nan)
# dum = np.where((ds.lon>lonrange[0], ds.lon<lonrange[1]), ds.smap_sss, np.nan)
plt.plot(dumx[0,:,:],dumy[0,:,:],'.')
# dum.shape

plt.show()

In [ ]:
# subset data spatially
# ds.where(ds.lon>lonrange[0] and ds.lat<lonrange[1])
# ds.where(ds.lon>lonrange[0], smap_sss)
# xr.where(ds.lon>lonrange[0], ds.smap_sss, 1).plot()
# ds.lon.plot()
# ds.sel(lon>lonrange[0])

# ideep = ( subsampled_data.dep.values == subsampled_data.dep.values.min() )
# this_var_onedep = sdata[vbl].where(sdata.dep == zpl, drop=True)


i = ( ds.lon.values>lonrange[0] )
plt.plot(ds.lon.sel(

In [ ]:


# from pyresample import geometry, geo_filter
lons = np.array([-170, -30, 30, 170])
lats = np.array([20, -40, 50, -80])
swath_def = geometry.SwathDefinition(lons, lats)

data = np.array([1, 2, 3, 4])
filter_area = geometry.AreaDefinition('test', 'test', 'test',
                                      {'proj' : 'eqc', 'lon_0' : 0.0, 'lat_0' : 0.0},
                                      8, 8,
                                      (-20037508.34, -10018754.17, 20037508.34, 10018754.17)
                                     )
filter = np.array([[1, 1, 1, 1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                  ])
grid_filter = geo_filter.GridFilter(filter_area, filter)
swath_def_filtered, data_filtered = grid_filter.filter(swath_def, data)


In [ ]:
# MODIS SST

provider = 'POCLOUD'
url = 'https://cmr.earthdata.nasa.gov/search/collections.umm_json'
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                            'page_size': 300,
                            'temporal': date_range 
                        },
                        headers={
                            'Accept': 'application/json'
                        }
                       )
response.headers['cmr-hits']



In [ ]:
# Loop through each response in the catalog and print the respective concept ID
# for r in response.json()['items']:
#     print('{} ==> '.format(r['meta']['s3-links'][0].split('/')[1]), r['meta']['concept-id'])

In [ ]:
# Loading credentials for direct access
def begin_s3_direct_access(url: str="https://archive.podaac.earthdata.nasa.gov/s3credentials"):
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})

fs = begin_s3_direct_access()

type(fs)

In [ ]:
url = 'https://cmr.earthdata.nasa.gov/search/granules'
response = requests.get(url, 
                        params={
                            'concept_id': 'C2036880650-POCLOUD',
                            'temporal': date_range,
                            'bounding_box': bounding_box, 
                            'page_size': 200,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
print(response)
print(response.headers['CMR-Hits'])

In [ ]:
granules_url = []
for gran in response.json()['feed']['entry']:
    granules_url.append(gran['links'][0]['href'])
granules_url

In [ ]:
# if the same file exists in NRT and non-NRT (ie, DT), skip the NRT version (there is definitely a cleaner way to do this...)
for gran in granules_url:
    if 'NRT' in gran and (gran.rsplit('NRT.nc', 1)[0] + 'nc') in granules_url: # if this has NRT *and* a non-NRT version exists
        granules_url.remove(gran)
print(granules_url)

In [ ]:
def extract_time(ds):
    t0 = np.datetime64(pd.to_datetime(ds.time_coverage_start).date())
    return ds.expand_dims(time=[t0])

In [ ]:
# load file(s)
# use sel and slice to get our region. MODIS SSTs have lat from + to -, so put the bigger lat value first in the slice
file_list =  [fs.open(file) for file in granules_url]
ds_modis = xr.open_mfdataset(file_list, preprocess=extract_time, engine='h5netcdf',concat_dim='time', combine='nested').sel(lon=slice(lonrange[0], lonrange[1]), lat=slice(latrange[1], latrange[0]))
ds_modis


### In situ data from UpTempO buoys

In [ ]:
# for now, just pick a single buoy & download it locally using wget (if new)
buoy_url = 'http://psc.apl.washington.edu/UpTempO/WebDATA/'

# from pyresample import geometry, geo_filter
lons = np.array([-170, -30, 30, 170])
lats = np.array([20, -40, 50, -80])
swath_def = geometry.SwathDefinition(lons, lats)

data = np.array([1, 2, 3, 4])
filter_area = geometry.AreaDefinition('test', 'test', 'test',
                                      {'proj' : 'eqc', 'lon_0' : 0.0, 'lat_0' : 0.0},
                                      8, 8,
                                      (-20037508.34, -10018754.17, 20037508.34, 10018754.17)
                                     )
filter = np.array([[1, 1, 1, 1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1, 1, 1, 1],
                  ])
grid_filter = geo_filter.GridFilter(filter_area, filter)
swath_def_filtered, data_filtered = grid_filter.filter(swath_def, data)

buoy_file = 'UpTempO_2019_02_SIZRS-20211117.dat'
buoy_url + buoy_file
!wget -N {buoy_url + buoy_file} -P downloads/

In [ ]:
# for this file, we want to skip the first 35 lines (header). can later generalize this
filename = (f'downloads/{buoy_file}')
f = open(filename,'r')
# data = f.readlines()[35:]
data = f.readlines()[35:40] # start with a small # of rows
f.close()

# %DATA COLUMNS:
# % 0 = year
# % 1 = month
# % 2 = day
# % 3 = hour (GMT)
# % 4 = Latitude (N)
# % 5 = Longitude (E)
# % 6 = Ocean Pressure (dB) at Sensor #1(Nominal Depth = 20.0 m)
# % 7 = Ocean Pressure (dB) at Sensor #2(Nominal Depth = 40.0 m)
# % 8 = Ocean Pressure (dB) at Sensor #3(Nominal Depth = 60.0 m)
# % 9 = Temperature at nominal depth 0.0 (m)
# % 10 = Temperature at nominal depth 2.5 (m)
# % 11 = Temperature at nominal depth 5.0 (m)
# % 12 = Temperature at nominal depth 7.5 (m)
# % 13 = Temperature at nominal depth 10.0 (m)
# % 14 = Temperature at nominal depth 15.0 (m)
# % 15 = Temperature at nominal depth 20.0 (m)
# % 16 = Temperature at nominal depth 25.0 (m)
# % 17 = Temperature at nominal depth 30.0 (m)
# % 18 = Temperature at nominal depth 35.0 (m)
# % 19 = Temperature at nominal depth 40.0 (m)
# % 20 = Temperature at nominal depth 50.0 (m)
# % 21 = Temperature at nominal depth 60.0 (m)
# % 22 = Sea Level Pressure (mBar)
# % 23 = Battery Voltage (V)
# % 24 = Submergence Percent
# 2019 08 14 20.3500 72.013322 -149.817103 0.250 0.431 0.611 13.29 5.022276 3.85962 3.70969 3.468275 3.823188 3.630601 3.738509 3.441308 3.728554 3.573352 3.425621 3.545923 1011.2 12.8 0
# data[[:,1]]

df = pd.read_csv(filename, sep=' ', skiprows=35, names=["year", "month", "day", "hour", "lat", 
                                               "lon", "p20", "p40", "p60", "T0", "T2p5", 
                                               "T5", "T7p5", "T10", "T15", "T20", "T25", 
                                               "T30", "T35", "T40", "T50","T60", "SLP", "voltage", "sub_perc"],
                parse_dates= {"date" : ["year","month","day","hour"]})
df


In [ ]:
# not quite right but close!
# df['dates'] = pd.to_datetime({df['year'],df['month'], df['day']}, format='%Y%m%d')

plt.plot(df['l'],df['T0'])
plt.show()

In [ ]:
df.where(df['T0']!=-999)


# plt.scatter(df['lon'],df['lat'],c=df['T0'],vmin=-5, vmax=5)
# plt.colorbar()

In [ ]:
# plt.plot(data_vars[lon])
# plt.show()
# attrs['sss_smap']
# smap_l2 = xr.Dataset(data_vars)
# data_vars?

# data_vars (dict-like, optional) – A mapping from variable names to DataArray objects,
# Variable objects or to tuples of the form (dims, data[, attrs]) which can be used as arguments 
# to create a new Variable. Each dimension must have the same length in all variables in which it appears.

# The following notations are accepted:

#     mapping {var name: DataArray}

#     mapping {var name: Variable}

#     mapping {var name: (dimension name, array-like)}

#     mapping {var name: (tuple of dimension names, array-like)}

#     mapping {dimension name: array-like} (it will be automatically moved to coords, see below)

# Each dimension must have the same length in all variables in which it appears.


